In [64]:
import pandas as pd
import os 
import plotly.express as px
import pymysql
import awswrangler as wr
import boto3




In [87]:
#Query MySQL
connection = pymysql.connect(host='192.168.216.202',user='edw',password='tekedw@951',
                                 port = 3306,db='tboedw')

try:
    with connection.cursor(pymysql.cursors.DictCursor) as cursor:
        sql = """
                    SELECT booked_from,
                    agency_country_name,
                    country_name,
                    month(invoice_date) AS mnth,
                    YEAR(invoice_date) AS yr,
                        sum(bb.revenue - bb.cost_of_sales - bb.credit_card_costs - bb.make_payment_charges)AS nrm,
                        ifnull(sum(bb.revenue - bb.cost_of_sales - bb.credit_card_costs - bb.make_payment_charges),0)/ifnull(sum(gross_sales),0) AS nrm,
                    SUM(gross_sales)
                    FROM(
                    SELECT (ifnull(aa.Handling_Charges,0) + 
                    ifnull(aa.Upfront_Margin,0) + 
                    ifnull(aa.vat_input,0) + 
                    ifnull(aa.enett_comm,0) + 
                    ifnull(aa.gp_share_of_ian,0) + 
                    ifnull(aa.other_publish_income,0) +
                    ifnull(aa.installmentCharges,0) - 
                    ifnull(aa.vat_amount_usd,0) + 
                    IFNULL(hotel_comm,0) + 
                    IFNULL(coupon_discount,0)+
                    IFNULL(orc_comm,0)+
                    IFNULL(expedia_incentive,0)) AS revenue,
                    
                    aa.cost_of_sales,invoice_date,country_name,
                    aa.credit_card_costs,
                    aa.make_payment_charges,aa.agency_country_name,aa.gross_sales,booked_from
                    FROM( 
                    SELECT (IFNULL(nm.booker_incentive, 0) + IFNULL(nm.agent_plb, 0) + IFNULL(nm.gsa_cost, 0) + IFNULL(nm.post_checkout_comm, 0) + IFNULL(nm.affliate_commission, 0) + IFNULL(nm.techhouse_cost, 0)
                    + IFNULL(nm.cashback_expenses, 0) + IFNULL(nm.amadeus_comm, 0)) AS cost_of_sales,coupon_discount,
                    nm.hotel_country AS country_name,
                    DATE(invoice_date) AS invoice_date,
                    (IFNULL(nm.cc_charges, 0) + IFNULL(nm.forex_anticipation, 0) + IFNULL(nm.direct_payment_comm, 0)) AS credit_card_costs,
                    ifnull(nm.orc_comm,0) AS orc_comm, nm.hotel_comm,
                    nm.make_payment_charges,a.agency_country_name,nm.gross_sales,
                    CASE
                                    WHEN (nm.gross_sales - nm.markup_supplier_price_usd_n ) < 0 THEN (nm.gross_sales -nm.markup_supplier_price_usd_n )
                                    ELSE 0
                                END AS Handling_Charges,
                                CASE
                                    WHEN (nm.gross_sales - nm.markup_supplier_price_usd_n ) >= 0 THEN (nm.gross_sales - nm.markup_supplier_price_usd_n)
                                    ELSE 0
                                END AS Upfront_Margin,nm.expedia_incentive,
                        nm.vat_input , nm.enett_comm , nm.gp_share_of_ian , nm.other_publish_income, nm.installmentCharges, nm.vat_amount_usd,booked_from    
                    FROM 
                    ( SELECT * , 
                    case when nm.invoice_type = 'Credit' AND nm.markup_supplier_price_usd >0 
                    then nm.markup_supplier_price_usd * -1 ELSE  nm.markup_supplier_price_usd END AS  markup_supplier_price_usd_n 
                    FROM stg_nrm_monitoring1 AS nm
                    WHERE date(nm.invoice_date) >= '2023-10-01'
                    AND DATE(nm.invoice_date)<= '2024-09-30' )nm
                    
                    LEFT JOIN stg_agency AS a ON nm.agency_id = a.agency_id
                    LEFT  JOIN stg_country AS c ON a.agency_country_name = c.country_name
                    #LEFT  JOIN stg_country cc ON nm.hotel_country = cc.country_name
                    )aa)bb
                    GROUP BY  
                    agency_country_name,booked_from,
                    country_name,
                    month(invoice_date),
                    YEAR(invoice_date)
                                    """

 

        cursor.execute(sql)
        result = cursor.fetchall()
finally:
    connection.close()

df_nrm = pd.DataFrame(result)

In [88]:
df_nrm

,booked_from,agency_country_name,country_name,mnth,yr,nrm,.nrm,SUM(gross_sales)
0,API,Kuwait,USA,10,2023,1178.889879,0.036890,31957.0500000000000000
1,API,USA,USA,10,2023,74266.869521,0.028804,2578366.8200000000000000
2,API,Canada,USA,10,2023,106598.046908,0.026720,3989401.7500000000000000
3,API,USA,Canada,10,2023,3.208671,0.000060,53263.0400000000000000
4,API,Ireland,USA,10,2023,14430.635686,0.037673,383054.3900245745014024
...,...,...,...,...,...,...,...,...
70842,B2B,Albania,Monaco,9,2024,88.674840,0.049898,1777.1048399901899815
70843,B2B,United Arab Emirates,Lesotho,9,2024,12.872250,0.067102,191.8300000000000000
70844,API,Chile,Ecuador,9,2024,80.037600,0.068934,1161.0800000000000000
70845,B2B,Brazil,Nigeria,9,2024,87.397908,0.100655,868.2898500009551030


In [85]:
#Query Athena

df_sb = wr.athena.read_sql_query(
    sql="""with searches AS (
    SELECT case when request_from = 1 then 'B2B' else 'API'
    end as booked_from,
        SUM(ss.search_cnt) AS searches,
        a.agency_country_name AS agency_country,
        c.country_name AS destination_country,
        MONTH(ss.searched_date) AS mnth,
        YEAR(ss.searched_date) AS yr
    FROM 
        "l2"."tboh_hotels_agg_booker_ctry_date_search" ss 
    INNER JOIN 
        "l1"."common_static_country" c ON
        lower(ss.search_country) =lower(c.country_code)
    INNER JOIN 
        "l1"."tboh_static_agency" a ON ss.agency_id = a.agency_id
    WHERE 
        CAST(ss.searched_date AS DATE) BETWEEN DATE '2023-10-01' AND DATE '2024-09-30'
    GROUP BY 1,3,4,5,6
        ),
    booking as(    
     SELECT 
    MONTH(voucher_date) AS mnth,
    YEAR(voucher_date) AS yr,
    COUNT(DISTINCT hotel_id) AS booking,
    a.agency_country_name AS agency_country,
    c.country_name AS destination_country,
    case when coalesce(r.booked_from,'B2B')='B2B' THEN 'B2B'
    ELSE 'API' end as booked_from 
FROM 
    "l1".tboh_hotels_fact_room_ro r 
INNER JOIN 
    "l1"."common_static_country" c ON 
    lower(r.destination_countrycode) = lower(c.country_code)
INNER JOIN 
    "l1"."tboh_static_agency" a ON r.agency_id = a.agency_id
WHERE 
    CAST(voucher_date AS DATE) BETWEEN DATE '2023-10-01' AND DATE '2024-09-30'
    AND r.voucher_status = 'Vouchered'
    -- AND a.agency_country_name <> 'India'
    -- AND r.booking_status = 'Confirmed'
GROUP BY 
    a.agency_country_name, 
    case when coalesce(r.booked_from,'B2B') = 'B2B'THEN 'B2B'
    ELSE 'API' end,
    c.country_name,
    MONTH(voucher_date),
    YEAR(voucher_date)
)
Select s.mnth, s.yr,s.booked_from, s.agency_country,
s.destination_country,
s.searches,
b.booking
from searches as s
inner join booking as b
on lower(s.agency_country) =lower( b.agency_country)
and lower(s.destination_country) =lower(b.destination_country)
and s.mnth = b.mnth
and s.yr = b.yr
and lower(s.booked_from) = lower(b.booked_from) """,
    database="l1",
    ctas_approach=False,
    boto3_session=boto3.session.Session(region_name="eu-west-1"),
)

In [97]:
df_sb['agency_country'] = df_sb['agency_country'].apply(lambda x : str(x).lower())
df_sb['destination_country'] = df_sb['destination_country'].apply(lambda x : str(x).lower())
df_sb['Year_month'] = df_sb['yr']*100 + df_sb['mnth']
df_sb['Source_dest'] = df_sb['agency_country'] + "-"+df_sb['destination_country']
df_sb['L2B'] = df_sb['booking']/df_sb['searches'] * 10000
df_sb.head()

,mnth,yr,booked_from,agency_country,destination_country,searches,booking,Year_month,Source_dest,L2B
0,3,2024,API,qatar,mauritius,285,2,202403,qatar-mauritius,70.175439
1,3,2024,B2B,oman,malaysia,160,16,202403,oman-malaysia,1000.0
2,4,2024,B2B,south africa,egypt,393,27,202404,south africa-egypt,687.022901
3,6,2024,API,poland,canada,236,4,202406,poland-canada,169.491525
4,9,2024,B2B,jordan,united kingdom,52,1,202409,jordan-united kingdom,192.307692


In [92]:
df_nrm['agency_country'] = df_nrm['agency_country_name'].apply(lambda x : str(x).lower())
df_nrm['destination_country'] = df_nrm['country_name'].apply(lambda x : str(x).lower())
df_nrm['Year_month'] = df_nrm['yr']*100 + df_nrm['mnth']
df_nrm['Source_dest'] = df_nrm['agency_country'] + "-"+df_nrm['destination_country']
df_nrm.columns = ['booked_from', 'agency_country_name', 'country_name', 'mnth', 'yr',
       'nrm', 'NRM', 'sales', 'agency_country',
       'destination_country', 'Year_month', 'Source_dest']
df_nrm.head()


,booked_from,agency_country_name,country_name,mnth,yr,nrm,.nrm,SUM(gross_sales),agency_country,destination_country,Year_month,Source_dest
0,API,Kuwait,USA,10,2023,1178.889879,0.036890,31957.0500000000000000,kuwait,usa,202310,kuwait-usa
1,API,USA,USA,10,2023,74266.869521,0.028804,2578366.8200000000000000,usa,usa,202310,usa-usa
2,API,Canada,USA,10,2023,106598.046908,0.026720,3989401.7500000000000000,canada,usa,202310,canada-usa
3,API,USA,Canada,10,2023,3.208671,0.000060,53263.0400000000000000,usa,canada,202310,usa-canada
4,API,Ireland,USA,10,2023,14430.635686,0.037673,383054.3900245745014024,ireland,usa,202310,ireland-usa


In [98]:
df = pd.merge(df_nrm[['Source_dest', 'Year_month', 'booked_from','NRM', 'sales']], df_sb[['Source_dest', 'Year_month', 'booked_from','L2B', 'booking']], on=['Source_dest', 'Year_month', 'booked_from'], how = 'inner' )

In [110]:
df

,Source_dest,Year_month,booked_from,NRM,sales,L2B,booking
0,indonesia-indonesia,202402,B2B,0.029618,233192.9672113645741264,2258.990376,1338
1,turkey-united kingdom,202402,API,0.047515,52967.0300000000000000,1.581454,118
2,greece-germany,202402,B2B,0.066357,123944.3365989168954058,5076.923077,132
3,ireland-usa,202402,API,0.039145,293129.4044187056552443,1.407027,875
4,usa-usa,202402,B2B,0.063421,832042.0600000000000000,583.554377,1276
...,...,...,...,...,...,...,...
46950,albania-monaco,202409,B2B,0.049898,1777.1048399901899815,1250.0,1
46951,united arab emirates-lesotho,202409,B2B,0.067102,191.8300000000000000,1428.571429,1
46952,chile-ecuador,202409,API,0.068934,1161.0800000000000000,69.93007,1
46953,brazil-nigeria,202409,B2B,0.100655,868.2898500009551030,454.545455,1


In [109]:
df.to_excel("Sector_nrm_l2b.xlsx")

In [106]:
df_api = df.loc[df['booked_from'] == 'API', :]
df_b2b = df.loc[df['booked_from'] == 'B2B', :]

df_api = df_api.loc[df_api['booking'] > 100, :]
df_b2b = df_b2b.loc[df_b2b['booking'] > 100, :]

In [107]:
fig = px.scatter(
    df_api,
    x='L2B',
    y='NRM',             
    hover_name='Source_dest',    
    animation_frame='Year_month',       
    size_max=20, 
    width = 800,
    height = 800,

)

fig.update_layout(
    title="Source-Destination L2B vs NRM - API",
    xaxis_title="L2B",
    yaxis_title="NRM",
    sliders=dict(
        title="Time",
        steps=[{'label': str(t), 'method': 'animate', 'args': [[t]]} for t in df['Year_month']]
    )
)

fig.show()

In [108]:
fig = px.scatter(
    df_b2b,
    x='L2B',
    y='NRM',             
    hover_name='Source_dest',    
    animation_frame='Year_month',       
    size_max=20, 
    width = 800,
    height = 800

)

fig.update_layout(
    title="Source-Destination L2B vs NRM - B2B",
    xaxis_title="L2B",
    yaxis_title="NRM",
    sliders=dict(
        title="Time",
        steps=[{'label': str(t), 'method': 'animate', 'args': [[t]]} for t in df['Year_month']]
    )
)

fig.show()